# 6. 19대 대선 결과 분석

## 1. Selenium과 Beautiful Soup을 이용한 데이터 획득 준비 작업

In [2]:
import pandas as pa
import numpy as np
import platform
import matplotlib.pyplot as plt 
%matplotlib inline

In [3]:
import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else:
    print('Unknown System..')

In [4]:
from selenium import webdriver
import time

In [5]:
driver = webdriver.Chrome('../driver/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

In [6]:
driver.find_element_by_id('electionType1').click()

In [7]:
driver.find_element_by_id('electionName').send_keys('제19대')

In [9]:
driver.find_element_by_id('electionCode').send_keys('대통령선거')

In [10]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name('option')
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

## 2. 19대 대선 개표 결과 데이터 획득하기

In [11]:
# '득표수(득표율)'에서 '(' 기준으로 왼쪽 숫자만 얻어서 콤마 제거하고 float형으로 변경하는 함수
import re
def get_num(tmp):
    return float(re.split('\(', tmp)[0].replace(',',''))

In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

wait = WebDriverWait(driver, 10)

def move_sido(name):
    element = driver.find_element_by_id('cityCode')
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH, make_xpath)))
    driver.find_element_by_xpath(make_xpath).click()